First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [8]:
ltw = pd.read_csv(f'{path}/data/ltw_treat.csv', encoding = 'UTF-8-SIG', sep=',', converters={'AGS': str})
ltw.head(10)

,AGS,GEN,year,Land,state_ID,Kreis,post_2010,post_2014,Union,SPD,...,Linke,Andere,treatment_05,treatment_09,treatment_13,treatment_30,treatment_60,treatment_100,ever_treated,never_treated
0,03101000,Braunschweig,2008.0,NI,3,3101,0.0,0.0,36.861503,31.245464,...,9.549529,3.232192,30.0,30.0,100.0,2005.0,2005.0,2005.0,30,1.0
1,03101000,Braunschweig,2013.0,NI,3,3101,1.0,0.0,29.666789,33.671272,...,4.246117,5.556723,30.0,30.0,100.0,2005.0,2005.0,2005.0,30,1.0
2,03101000,Braunschweig,2017.0,NI,3,3101,1.0,1.0,26.342641,37.604703,...,6.741527,9.544639,30.0,30.0,100.0,2005.0,2005.0,2005.0,30,1.0
3,03102000,Salzgitter,2008.0,NI,3,3102,0.0,0.0,37.476063,39.268958,...,8.828035,4.394868,30.0,30.0,100.0,2005.0,2005.0,2005.0,30,1.0
4,03102000,Salzgitter,2013.0,NI,3,3102,1.0,0.0,30.880127,44.075623,...,4.135136,5.602442,30.0,30.0,100.0,2005.0,2005.0,2005.0,30,1.0
5,03102000,Salzgitter,2017.0,NI,3,3102,1.0,1.0,24.445995,45.203436,...,4.862732,15.901927,30.0,30.0,100.0,2005.0,2005.0,2005.0,30,1.0
6,03103000,Wolfsburg,2008.0,NI,3,3103,0.0,0.0,43.121141,31.515784,...,7.863727,4.298200,60.0,60.0,NaN,0.0,2005.0,2005.0,60,1.0
7,03103000,Wolfsburg,2013.0,NI,3,3103,1.0,0.0,37.629895,35.812910,...,3.210075,5.103196,60.0,60.0,NaN,0.0,2005.0,2005.0,60,1.0
8,03103000,Wolfsburg,2017.0,NI,3,3103,1.0,1.0,29.942107,43.211115,...,4.525280,10.474720,60.0,60.0,NaN,0.0,2005.0,2005.0,60,1.0
9,03151001,Adenbüttel,2008.0,NI,3,3151,0.0,0.0,47.568209,30.960854,...,4.863582,3.202847,30.0,30.0,100.0,2005.0,2005.0,2005.0,30,1.0


View summary statistics

In [4]:
columns = ['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
ltw.describe().to_csv(f'{path}/tables/ltw/ltw_descriptives.csv', sep=';', encoding='utf-8-sig', decimal=',', columns=columns, float_format='%.2f')
ltw[columns].describe()

,Union,SPD,FDP,Linke,Grüne,Andere
count,16734.000000,16734.000000,16734.000000,16734.000000,16734.000000,16734.000000
mean,39.951643,18.747696,6.090268,10.265374,7.748091,17.238718
std,11.413896,11.211236,3.694388,11.175634,4.863212,10.437317
min,5.797101,0.000000,0.000000,0.202429,0.000000,0.000000
25%,31.862229,9.657948,3.333617,2.402541,4.073789,8.396709
50%,39.149065,16.061681,5.485893,4.157848,6.424154,16.193220
75%,47.347304,26.347302,7.894737,21.129012,10.272994,24.731183
max,85.300668,58.673469,55.555556,47.500000,38.003901,71.186441


In [24]:
ltw['year'].value_counts().to_csv(f'{path}/tables/ltw/ltw_obsyears.csv', sep=';', encoding='utf-8-sig', decimal=',', float_format='%.2f')

Calculate average vote shares in all years by treated 50

In [9]:
avg_dfs = {2010: 'treatment_09', 2014: 'treatment_13'}
for year, treatment in avg_dfs.items():
    ltw[treatment] = ltw[treatment].astype('Int64').astype(str)
    ltw[treatment] = np.where(ltw[treatment].isna(), 'None', ltw[treatment])
    avg_dfs[year] = ltw[['Land','Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', treatment, 'year']].groupby(['Land', treatment, 'year'], as_index = False).mean()
    avg_dfs[year].rename(columns = {f'{treatment}':'treatment'}, inplace = True)
avg_dfs[2010].head(15)

,Land,treatment,year,Union,SPD,FDP,Linke,Grüne,Andere
0,BY,100,2008.0,48.928283,15.867770,6.653374,4.478636,5.899769,18.297109
1,BY,100,2013.0,50.372369,17.766905,2.993741,2.200036,6.510306,20.281161
2,BY,100,2018.0,44.035370,9.183620,3.844595,2.592355,11.655057,28.813386
3,BY,30,2008.0,47.499765,26.180209,4.731673,3.919696,4.072898,13.709806
4,BY,30,2013.0,47.310362,26.723011,2.080642,1.708022,4.565271,17.694892
5,BY,30,2018.0,43.040510,17.057659,3.328715,2.586176,9.251469,24.837251
6,BY,60,2008.0,48.924059,20.326455,5.532564,4.299898,5.178007,15.853728
7,BY,60,2013.0,49.904328,22.566043,2.309538,1.958787,5.894380,17.478662
8,BY,60,2018.0,44.702568,13.377289,3.529717,2.370771,11.012472,25.141409
9,BY,<NA>,2008.0,47.296932,14.174480,6.979073,3.712867,7.846300,20.104965


For each state, plot the average vote shares over years for treated and untreated municipalities

In [11]:
states = {'BY': 'Bavaria', 'HE': 'Hesse', 'NW': 'North-Rhine Westphalia', 'NI': 'Lower Saxony', 'TH': 'Thuringia'}
treatments = ['None', '30', '60', '100']
blacks = ['#000000', '#191919', '#323232', '#4c4c4c']
reds = ['#cc0000', '#d11919', '#d63232', '#db4c4c']
yellows = ['#ffff00', '#ffff19', '#ffff32', '#ffff4c']
purples = ['#800080', '#8c198c', '#993299', '#a64ca6']
greens = ['#008000', '#198c19', '#329932', '#4ca64c']
greys = ['#808080', '#8c8c8c', '#999999', '#a6a6a6']
markers = ['o', 's', 'v', 'D']
for year, df in avg_dfs.items():
    for abb, state in states.items():
        # keep only obs. corresponding to state
        state_data = df[df['Land'] == abb]
        # get years in a list
        years = state_data.year.unique().astype(int).tolist()
        # initialize figure 
        fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), dpi=300)
        # set x labels
        plt.setp(axes, xticks=years)
        # set legend font size
        plt.rc('legend',fontsize=6)
        # title for entire figure
        fig.suptitle(f'Average Vote Shares \n {state} State Parliament', fontsize=20)

        for i, treatment in enumerate(treatments):
            plot_data = state_data[state_data['treatment'] == treatment]
            # edit subplots
            axes[0, 0].set_title('Union', fontsize=12)
            axes[0, 0].set_ylabel('% Votes', fontsize=10)
            axes[0, 0].plot(plot_data['year'], plot_data['Union'], color=blacks[i], marker = markers[i], label=treatment, lw=0.8, markersize=4)
            axes[0, 0].legend(loc='lower left')
            axes[0, 0].axvline(x=year, color='red', linestyle='--')

            axes[0, 1].set_title('SPD', fontsize=12)
            axes[0, 1].set_ylabel('% Votes', fontsize=10)
            axes[0, 1].plot(plot_data['year'], plot_data['SPD'], color=reds[i], marker = markers[i], label=treatment, lw=0.8, markersize=4)
            axes[0, 1].legend(loc='lower left')
            axes[0, 1].axvline(x=year, color='red', linestyle='--')

            axes[1, 0].set_title('FDP', fontsize=12)
            axes[1, 0].set_ylabel('% Votes', fontsize=10)
            axes[1, 0].plot(plot_data['year'], plot_data['FDP'], color=yellows[i], marker = markers[i], label=treatment, lw=0.8, markersize=4)
            axes[1, 0].legend(loc='upper left')
            axes[1, 0].axvline(x=year, color='red', linestyle='--')

            axes[1, 1].set_title('Linke', fontsize=12)
            axes[1, 1].set_ylabel('% Votes', fontsize=10)
            axes[1, 1].plot(plot_data['year'], plot_data['Linke'], color=purples[i], marker = markers[i], label=treatment, lw=0.8, markersize=4)
            axes[1, 1].legend(loc='upper left')
            axes[1, 1].axvline(x=year, color='red', linestyle='--')

            axes[2, 0].set_title('Grüne', fontsize=12)
            axes[2, 0].set_ylabel('% Votes', fontsize=10)
            axes[2, 0].plot(plot_data['year'], plot_data['Grüne'], color=greens[i], marker = markers[i], label=treatment, lw=0.8, markersize=4)
            axes[2, 0].legend(loc='upper left')
            axes[2, 0].axvline(x=year, color='red', linestyle='--')

            axes[2, 1].set_title('Andere', fontsize=12)
            axes[2, 1].set_ylabel('% Votes', fontsize=10)
            axes[2, 1].plot(plot_data['year'], plot_data['Andere'], color=greys[i], marker = markers[i], label=treatment, lw=0.8, markersize=4)
            axes[2, 1].legend(loc='upper left')
            axes[2, 1].axvline(x=year, color='red', linestyle='--')

        # Save figure
        fig.savefig(f'{path}/figures/ltw/ltw_{abb}_{year}_vote_shares.png')
        plt.close()

Plot a map with the partys who received the largest first vote share in each year for each municipality

In [19]:
# Load shapefiles
import geopandas as gpd
municipalities = gpd.read_file(f'{path}/data/geodata/municipalities.shp')
states = gpd.read_file(f'{path}/data/geodata/VG250_LAN.shp')
powerlines = gpd.read_file(f'{path}/data/geodata/all_powerlines.shp')
powerlines = powerlines.set_geometry('geometry')

Replace State IDs with Abbreviations 

In [20]:
abb = {'01': 'SH', '02': 'HH', '03': 'NI', '04': 'HB', '05': 'NW', '06': 'HE', '07': 'RP', '08': 'BW', \
         '09': 'BY', '10': 'SL', '11': 'BE', '12': 'BB', '13': 'MV', '14': 'SN', '15': 'ST', '16': 'TH'}
for key, value in abb.items():
    municipalities['SN_L'] = np.where(municipalities['SN_L'] == key, value, municipalities['SN_L'])
    states['SN_L'] = np.where(states['SN_L'] == key, value, states['SN_L'])

In [21]:
# Prepare df with election winners
ltw_win = ltw[['AGS', 'Land', 'year', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
ltw_win['winner'] = ltw_win[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']].idxmax(axis = 1)
ltw_win = ltw_win[['AGS', 'Land', 'year', 'winner']]
ltw_win['winner'].describe()

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_16751/1152327886.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltw_win['winner'] = ltw_win[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']].idxmax(axis = 1)


count     16734
unique        6
top       Union
freq      12435
Name: winner, dtype: object

Add colors

In [22]:
# add color column to df
colors = {'Union': '#000000', 'SPD': '#cc0000', 'FDP': '#ffff00', 'Grüne': '#008000', 'Linke': '#800080', 'Andere': '#808080'}
ltw_win['color'] = 0
for party, color in colors.items():
    ltw_win['color'] = np.where(ltw_win['winner'] == party, color, ltw_win['color'])

In [23]:
# Merge win and municipalities
ltw_win_plot = pd.merge(ltw_win, municipalities, on='AGS')
ltw_win_plot = ltw_win_plot.set_geometry('geometry')

In [24]:
from geopandas.tools import overlay
abb = {'BY': 'Bavaria', 'HE': 'Hesse', 'NW': 'North-Rhine Westphalia', 'NI': 'Lower Saxony', 'TH': 'Thuringia'}

for abb, state in abb.items():
    # keep only obs. corresponding to state
    state_winners = ltw_win_plot[ltw_win_plot['Land'] == abb]
    municipalities_plot = municipalities[municipalities['SN_L'] == abb]
    states_plot = states[states['SN_L'] == abb]
    # keep only parts of powerlines that intersect state
    powerlines_state = powerlines.overlay(states_plot, how='intersection')
    # get years in a list
    years = state_winners.year.unique().astype(int).tolist()
    # initialize figure 
    fig, ax = plt.subplots(nrows=3, figsize=(10, 10), dpi=300)
    # title for entire figure
    fig.suptitle(f'{state} State Parliament Election Winners', fontsize=20)
    for i, year in enumerate(years):
        plot_data = state_winners[state_winners['year'] == year]
        ax[i].set_aspect('equal')
        ax[i].set_axis_off()
        ax[i].set_title(f'{year}')
        municipalities_plot.plot(ax=ax[i], color='lightblue', edgecolor='blue', lw=0.01, zorder=1)
        states_plot.boundary.plot(ax=ax[i], color='darkblue', lw = 0.1, zorder=2)
        plot_data.plot(ax=ax[i], color=plot_data['color'], lw=0.01, zorder=1)
        powerlines_state.plot(ax=ax[i], column='year', cmap='viridis', legend=True, lw=2, zorder=2)
    plt.savefig(f'{path}/figures/ltw/ltw_{abb}_winners.png')
    plt.close()